In [7]:
import plotly.graph_objects as go
from ipywidgets import interact



In [25]:
json_input = """
{
"intrinsics": [
            {
                "camera_type": "kb4",
                "intrinsics": {
                    "fx": 325,
                    "fy": 325,
                    "cx": 423,
                    "cy": 891,
                    "k1": 0.04515034232861797,
                    "k2": 0.0375057118375968,
                    "k3": -0.01771880641236273,
                    "k4": 0.0029992584412460536
                }
            },
            {
                "camera_type": "kb4",
                "intrinsics": {
                    "fx": 325,
                    "fy": 325,
                    "cx": 423,
                    "cy": 891,
                    "k1": 0.33620592877651497,
                    "k2": 0.12115250811553866,
                    "k3": -0.1876453365159163,
                    "k4": 0.061278232686206839
                }
            },
            {
                "camera_type": "eucm",
                "intrinsics": {
                    "fx": 325,
                    "fy": 325,
                    "cx": 423,
                    "cy": 891,
                    "alpha": 0.6493410193202258,
                    "beta": 1.0723193423318838
                }
            },
            {
                "camera_type": "eucm",
                "intrinsics": {
                    "fx": 325,
                    "fy": 325,
                    "cx": 423,
                    "cy": 891,
                    "alpha": 0.5751897096633911,
                    "beta": 0.8913038372993469
                }
            },
            {
                "camera_type": "pinhole",
                "intrinsics": {
                    "fx": 325,
                    "fy": 325,
                    "cx": 423,
                    "cy": 891
                }
            }
        ]
}
"""

labels= ["near-perfect kb4", "+ve kb4", "old eucm", "new eucm", "pinhole"]

In [26]:
import json
import distortion_models

def load_model_from_json(json_input):
    data = json.loads(json_input)

    N = len(data['intrinsics'])
    cameras = list()
    
    for i in range(N):
        type = data['intrinsics'][i]['camera_type']
        label = labels[i]
        print(f"camera {i} has type {type}, label {label}")

        cameras.append(distortion_models.camera.get_camera_from_dict(data['intrinsics'][i]))

    return cameras


cameras = load_model_from_json(json_input)
print(len(cameras))

camera 0 has type kb4, label near-perfect kb4
camera 1 has type kb4, label +ve kb4
camera 2 has type eucm, label old eucm
camera 3 has type eucm, label new eucm
camera 4 has type pinhole, label pinhole
5


### Visualising R-theta

In [27]:
# Generating 3D points per degree of theta
# https://arxiv.org/pdf/1905.01489v1.pdf

import numpy as np

degs = np.linspace(0, 90, 90, endpoint=False)
rads = degs / 180 * np.pi

xs = np.sin(rads)
ys = np.zeros(len(rads))
zs = np.cos(rads)

points_3d = np.column_stack((xs, ys, zs))

print(points_3d.shape)

points_2d_list = []
                    
for camera in cameras:
    camera.cx = 0
    camera.cy = 0
    points_2d, valid = camera.project(points_3d)
    points_2d_list.append(points_2d)


(90, 3)


In [28]:
fig = go.Figure()

for i in range(len(points_2d_list)):
    points_2d = points_2d_list[i]
    print(points_2d.shape)
    r = np.linalg.norm(points_2d, axis=-1)
    print(r.shape)
    fig.add_trace(go.Scatter(x=degs, y=r, mode='lines+markers', name=f"#{i} {labels[i]}"))

fig.update_layout(yaxis_range=[-20,880])
fig.update_layout(height=900, width=960)
fig.show()

(90, 2)
(90,)
(90, 2)
(90,)
(90, 2)
(90,)
(90, 2)
(90,)
(90, 2)
(90,)


In [13]:
fig = go.FigureWidget()
bar = fig.add_bar(x=['x', '1-x'])
fig.layout = dict(yaxis=dict(range=[0,1]), height=600)

@interact(x=(0, 1, 0.01))
def update(x=0.3):
    with fig.batch_update():
        fig.data[0].y=[x, 1-x]
fig

interactive(children=(FloatSlider(value=0.3, description='x', max=1.0, step=0.01), Output()), _dom_classes=('w…

FigureWidget({
    'data': [{'type': 'bar', 'uid': 'a50b48c5-db71-414b-8596-889fa35f90fe', 'x': ['x', '1-x'], 'y': [0.3, 0.7]}],
    'layout': {'height': 600, 'template': '...', 'yaxis': {'range': [0, 1]}}
})